<a href="https://colab.research.google.com/github/pamisha/hgi/blob/main/Copy_of_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas scikit-learn surprise

import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

# Download and unzip the MovieLens dataset
!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip

# Load the dataset into a pandas DataFrame
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Define a Reader object with the expected format of the dataset
reader = Reader(rating_scale=(1, 5))

# Load the dataset into Surprise
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Use the SVD algorithm for training
algo = SVD()

# Train the algorithm on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)

# Calculate and print RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

def get_top_n_recommendations(predictions, n=10):
    from collections import defaultdict

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the n highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Generate recommendations for all users
top_n_recommendations = get_top_n_recommendations(predictions, n=10)

# Print the top 10 recommendations for a specific user (user_id = 1)
user_id = 1
print(f"Top 10 recommendations for user {user_id}:")
for movie_id, predicted_rating in top_n_recommendations[user_id]:
    print(f"Movie ID: {movie_id}, Predicted Rating: {predicted_rating}")